# < Facial Verification >

- https://github.com/serengil/deepface

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
![deep_face](./data/jupyter_image/deepface.jpg)

- **Tensorflow/Keras** 기반 얼굴 인식 시스템
- VGG-Face, Facenet, OpenFace, ArcFace 등 다양한 얼굴 인식 모델 사용 가능
- opencv, ssd, dlib, mtcnn, retinaface, mediapipe 등 다양한 얼굴 검출 모델 사용 가능
- 감정, 나이 등의 얼굴 속성까지 구분 가능

In [ ]:
import cv2
import pandas as pd
from matplotlib import pyplot as plt
from deepface import DeepFace
from deepface.commons import functions, realtime, distance as dst

In [ ]:
# Prevent GPU Memory Error
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto, Session
from tensorflow.compat.v1.keras.backend import set_session
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction =0.3 #GPU 메모리 사용을 30%로 제한
session = Session(config=config)
set_session(session)

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
![deep_face_models](./data/jupyter_image/deepface_models.jpg)

In [ ]:
models = [
  "VGG-Face", #0
  "Facenet", #1
  "Facenet512", #2 
  "OpenFace", #3
  "DeepFace", #4
  "DeepID", #5
  "ArcFace", #6 (Use)
  "Dlib", #7
  "SFace", #8
]

In [ ]:
metrics = [
    "cosine", #0 (Use)
    "euclidean", #1 
    "euclidean_l2" #2
]

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
![deep_face_detectors](./data/jupyter_image/deepface_detectors.jpg)

In [ ]:
detector_backends = [
    'opencv', #0
    'ssd', #1
    'dlib', #2
    'mtcnn', #3
    'retinaface', #4 (Use)
    'mediapipe' #5
]

In [ ]:
def plotTwoImg(img1, img2, img1_title, img2_title, figsize=(10,10)) :
    #Plot Two Image
    fig = plt.figure(figsize=figsize)
    ax1 = fig.add_subplot(2,2,1)
    ax2 = fig.add_subplot(2,1,1)
    ax1.imshow(img1)
    ax2.imshow(img2)
    ax1.set_title(img1_title)
    ax2.set_title(img2_title)
    plt.show()

# 1. Face Verification - Manual

In [ ]:
def verificationByDistance(distance) :
    if distance <= 0.68 : # When use ArcFace
        return "same"
    else :
        return "different"

## 1) Use Face Detector & Extract Embeddings

### (1) Different Person

- **Read & Check Original Image** 

In [ ]:
img1_path = './data/verification/'
img2_path = './data/verification/'

img1_name = 'BlackPink_Jenny.jpg'
img2_name = 'BlackPink_JiSoo.jpg'

In [ ]:
img1 = cv2.cvtColor(cv2.imread(img1_path + img1_name),cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(img2_path + img2_name),cv2.COLOR_BGR2RGB)
plotTwoImg(img1, img2 ,img1_name, img2_name, (17,8))

**Detect Face (2-Images)**

> **Use Below Function**

        DeepFace.detectFace(
            img_path = ''
            target_size = (N,N)
            detector_backend = ''
        )

In [ ]:
img1_face = DeepFace.detectFace(
    img_path = img1_path + img1_name, 
    target_size = (112,112), #ArcFace Input Size
    detector_backend = detector_backends[4] #RetinaFace
) 

img2_face = DeepFace.detectFace(
    img_path = img2_path + img2_name, 
    target_size = (112,112), #ArcFace Input Size
    detector_backend = detector_backends[4] #RetinaFace
)

- **Check Detected Image Results**

In [ ]:
plotTwoImg(img1_face, img2_face , img1_name, img2_name, (12,5))

- **Extract Face Feature Map from each image**

> **Use Below Function**

        DeepFace.represent(
            img_path = '' or numpy.array(BGR)
            enforce_detection = True/False,
            detector_backend = ''
            model_name = ''
            align = True/False
            normalization = 'base' / 'raw' / 'Facenet' / 'Facenet2018' / 'VGGFace' / 'VGGFace2' / 'ArcFace'
        )

In [ ]:
img1 = cv2.cvtColor(img1,cv2.COLOR_RGB2BGR)
img2 = cv2.cvtColor(img2,cv2.COLOR_RGB2BGR)

In [ ]:
img1_Facefeature = DeepFace.represent(
    img_path = img1,
    enforce_detection = True,
    detector_backend = detector_backends[4], #RetinaFace
    model_name = models[6], #ArcFace
    align = True,
    normalization = 'ArcFace'
)

img2_Facefeature = DeepFace.represent(
    img_path = img2,
    enforce_detection = True,
    detector_backend = detector_backends[4], #RetinaFace
    model_name = models[6], #ArcFace
    align = True,
    normalization = 'ArcFace'
)

- **Calculate distance by Feature Maps**

In [ ]:
distance = dst.findCosineDistance(img1_Facefeature, img2_Facefeature)

In [ ]:
distance

- **Verification by Distance**

In [ ]:
verificationByDistance(distance)

### (2) Same Person

- **Read & Check Original Image**

In [ ]:
img1_path = './data/verification/'
img2_path = './data/verification/'

img1_name = 'BTS_V_1.jpg'
img2_name = 'BTS_V_2.jpg'

In [ ]:
img1 = cv2.cvtColor(cv2.imread(img1_path + img1_name),cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(img2_path + img2_name),cv2.COLOR_BGR2RGB)
plotTwoImg(img1, img2 ,img1_name, img2_name, (17,8))

- **Detect Face image(2-Images)**

In [ ]:
img1_face = DeepFace.detectFace(
    img_path = img1_path + img1_name, 
    target_size = (112,112), #ArcFace Input Size
    detector_backend = detector_backends[4] #RetinaFace
) 

img2_face = DeepFace.detectFace(
    img_path = img2_path + img2_name, 
    target_size = (112,112), #ArcFace Input Size
    detector_backend = detector_backends[4] #RetinaFace
)

- **Check Detected Image Results**

In [ ]:
plotTwoImg(img1_face, img2_face , img1_name, img2_name, (12,5))

- **Extract Face Feature Map from each image**

In [ ]:
img1_Facefeature = DeepFace.represent(
    img_path = img1,
    enforce_detection = True,
    detector_backend = detector_backends[4], #RetinaFace
    model_name = models[6], #ArcFace
    align = True,
    normalization = 'ArcFace'
)

img2_Facefeature = DeepFace.represent(
    img_path = img2,
    enforce_detection = True,
    detector_backend = detector_backends[4], #RetinaFace
    model_name = models[6], #ArcFace
    align = True,
    normalization = 'ArcFace'
)

- **Calculate distance by Feature Maps**

In [ ]:
distance = dst.findCosineDistance(img1_Facefeature, img2_Facefeature)

In [ ]:
distance

- **Verification by Distance**

In [ ]:
verificationByDistance(distance)

### (3) Similar Person

- **Read & Check Original Image** 

In [ ]:
img1_path = './data/verification/'
img2_path = './data/verification/'

img1_name = 'JangKeunSuk.jpg'
img2_name = 'LeeHongGi.jpg'

In [ ]:
img1 = cv2.cvtColor(cv2.imread(img1_path + img1_name),cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(img2_path + img2_name),cv2.COLOR_BGR2RGB)
plotTwoImg(img1, img2 ,img1_name, img2_name, (17,8))

- **Detect Face (2-Images)**

In [ ]:
#TO DO - Detect 2 Images
img1_face = 






img2_face = 








- **Check Detected Image Results**

In [ ]:
plotTwoImg(img1_face, img2_face , img1_name, img2_name, (12,5))

- **Extract Face Feature Map from each image**

In [ ]:
#TO DO Extract Feature
img1_Facefeature = 








img2_Facefeature = 









- **Calculate distance by Feature Maps**

In [ ]:
# TO DO - Calculate Distance
distance = 

In [ ]:
distance

- **Verification by Distance**

In [ ]:
#TO DO - Verification


### (4) Twin

- **Read & Check Original Image** 

In [ ]:
img1_path = './data/verification/'
img2_path = './data/verification/'

img1_name = 'Fred_Widsley.jpg'
img2_name = 'George_Widsley.jpg'

In [ ]:
img1 = cv2.cvtColor(cv2.imread(img1_path + img1_name),cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(img2_path + img2_name),cv2.COLOR_BGR2RGB)
plotTwoImg(img1, img2 ,img1_name, img2_name, (17,8))

- **Detect Face (2-Images)**

In [ ]:
img1_face = DeepFace.detectFace(
    img_path = img1_path + img1_name, 
    target_size = (112,112), 
    detector_backend = detector_backends[4] #RetinaFace
) 

img2_face = DeepFace.detectFace(
    img_path = img2_path + img2_name, 
    target_size = (112,112), 
    detector_backend = detector_backends[4] #RetinaFace
)

- **Check Detected Image Results**

In [ ]:
plotTwoImg(img1_face, img2_face , img1_name, img2_name, (12,5))

- **Extract Face Feature Map from each image**

In [ ]:
img1_Facefeature = DeepFace.represent(
    img_path = img1,
    enforce_detection = True,
    detector_backend = detector_backends[4], #RetinaFace
    model_name = models[6], #ArcFace
    align = True,
    normalization = 'ArcFace'
)

img2_Facefeature = DeepFace.represent(
    img_path = img2,
    enforce_detection = True,
    detector_backend = detector_backends[4], #RetinaFace
    model_name = models[6], #ArcFace
    align = True,
    normalization = 'ArcFace'
)

- **Calculate distance by Feature Maps**

In [ ]:
distance = dst.findCosineDistance(img1_Facefeature, img2_Facefeature)

In [ ]:
distance

- **Verification by Distance**

In [ ]:
verificationByDistance(distance)

## 2) Use DeepFace.verify()

> **Use Below Function**

        DeepFace.verify (
            img1_path = '' or numpy array(BGR)
            img2_path = '' or numpy array(BGR)
            model_name = ''
            distance_metric = ''
            model = model
            enforce_detecton = True/False
            detector_backend = ''
            align = True
            prog_bar =  True
            normalization = 'base' / 'raw' / 'Facenet' / 'Facenet2018' / 'VGGFace' / 'VGGFace2' / 'ArcFace'
            )

 - Face Detection 수행
 - Face Feature Map 추출
 - 두 이미지 Feature Map 간 Distance 계산 수행
 - Detection / Feature Map 추출 전 **다양한 전처리 수행**

### (1) Different Person

In [ ]:
result = DeepFace.verify(
        img1_path = './data/verification/BlackPink_Jenny.jpg',
        img2_path = './data/verification/BlackPink_JiSoo.jpg',
        model_name = models[6], #ArcFace
        distance_metric = metrics[0], #Cosiine
        enforce_detection = True,
        align = True,
        detector_backend = detector_backends[4], #RetinaFace
        normalization = 'ArcFace'
    )
result

### (2) Same Person

In [ ]:
result = DeepFace.verify(
        img1_path = './data/verification/BTS_V_1.jpg',
        img2_path = './data/verification/BTS_V_2.jpg',
        model_name = models[6], #ArcFace
        distance_metric = metrics[0], #Cosiine
        enforce_detection = True,
        align = True,
        detector_backend = detector_backends[4], #RetinaFace
        normalization = 'ArcFace'
    )
result

### (3) Similar Person

In [ ]:
result = DeepFace.verify(
        img1_path = './data/verification/JangKeunSuk.jpg',
        img2_path = './data/verification/LeeHongGi.jpg',
        model_name = models[6], #ArcFace
        distance_metric = metrics[0], #Cosiine
        enforce_detection = True,
        align = True,
        detector_backend = detector_backends[4], #RetinaFace
        normalization = 'ArcFace'
    )
result

### (4) Twin

In [ ]:
result = DeepFace.verify(
        img1_path = './data/verification/Fred_Widsley.jpg',
        img2_path = './data/verification/George_Widsley.jpg',
        model_name = models[6], #ArcFace
        distance_metric = metrics[0], #Cosiine
        enforce_detection = True,
        align = True,
        detector_backend = detector_backends[4], #RetinaFace
        normalization = 'ArcFace'
    )
result